In [8]:
import pickle
import numpy as np

from pathlib import Path

from datautilities import meshmethods as mm

In [9]:
nregions = [1, 2, 3]
region_pattern = "Region{}LR4"

test_years = 1964
res = "HR_6cpu"

root_dirs = [Path("../Data") / region_pattern.format(nreg) for nreg in nregions]


In [10]:
diffs = []
y_tests = []
shps = []
for nreg, root_dir in zip(nregions, root_dirs):
    pred_fname = f"region{nreg}-lr8-migraphnet-59-60-61-62-63-15-5-wreg-hs-dir.npy"
    pred = np.load(root_dir / "predictions" / pred_fname)

    drop_idx = np.load(root_dir / res / "drop_index.npy")
    y_test = mm.load_data(years=test_years, root_dir=root_dir, res=res)
    y_test = np.delete(y_test, drop_idx, axis=1)
    shps.append(y_test.shape)
    y_test = y_test.flatten()

    # diff = np.abs(pred - y_test)
    diff = np.abs(pred - y_test)/y_test
    diff[diff == np.inf] = np.nan
    diffs.append(diff)
    y_tests.append(y_test)

/home/jannik/Documents/PhD/Super_Resolution_General/SROnUnstructuredGrids/datautilities/datautilities/meshmethods.py:106: RuntimeWarning: invalid value encountered in cast
  data[idx] = np.load(fpath).squeeze()
/tmp/ipykernel_64357/866532592.py:15: RuntimeWarning: divide by zero encountered in divide
  diff = np.abs(pred - y_test)/y_test


# Prepare general boxplot data

In [11]:
# out_path = Path("box-data") / "box_list_graphnet.pkl"
out_path = Path("box-data") / "box_list_norm_graphnet.pkl"

if out_path.exists():
    general_box_list = pickle.load(out_path.open("rb"))
else:
    general_box_list = []
    for diff in diffs:
        diff = 100*diff  # Convert to cm
        box = {}
        box["med"] = np.nanmedian(diff)
        box["q1"] = np.nanpercentile(diff, 25)
        box["q3"] = np.nanpercentile(diff, 75)
        box["whislo"] = np.nanpercentile(diff, 5)
        box["whishi"] = np.nanpercentile(diff, 95)
        box["mean"] = np.nanmean(diff)

        general_box_list.append(box)

    pickle.dump(general_box_list, out_path.open("wb"))

# Prepare wave-dependent boxplot data

In [12]:
edges = np.arange(0, 4.6, 0.5)

errors_list = []
for idx, diff in enumerate(diffs):
    errors = []

    for i in range(len(edges) - 1):
        range_mask = (edges[i] <= y_tests[idx]) & (y_tests[idx] < edges[i+1])
        errors.append(diff[range_mask]*100)

    errors_list.append(errors)


In [6]:
# Average dependent on offshore distance
offshore_pnts = [139, 358, 33]
edges = np.arange(0, 4.6, 0.5)
errors_list = []
for idx, diff in enumerate(diffs):
    errors = []
    # Convert to %
    diff = 100*diff.reshape(shps[idx])
    y_test = y_tests[idx].reshape(shps[idx])
    for i in range(len(edges) - 1):
        range_mask = (edges[i] <= y_test[:, offshore_pnts[idx]]) & (y_test[:, offshore_pnts[idx]] < edges[i+1])
        errors.append(diff[range_mask].flatten())

    errors_list.append(errors)

In [17]:
a0 = np.array([len(errors) for errors in errors_list[0]])
a1 = np.array([len(errors) for errors in errors_list[1]])
a2 = np.array([len(errors) for errors in errors_list[2]])
print(a0/a0.sum())
print(a1/a1.sum())
print(a2/a2.sum())

[3.24912501e-01 4.02590209e-01 1.53743900e-01 6.76035977e-02
 3.00202707e-02 1.36196554e-02 6.17294262e-03 1.30146402e-03
 3.54598394e-05]
[3.13559424e-01 4.03030567e-01 1.63378482e-01 6.74109689e-02
 2.85950354e-02 1.53965813e-02 6.85495296e-03 1.74290397e-03
 3.10857342e-05]
[2.78652579e-01 4.18708989e-01 1.81681029e-01 6.94371458e-02
 2.88459601e-02 1.45477671e-02 6.72041552e-03 1.38583380e-03
 2.02811248e-05]


In [7]:
# out_path = Path("box-data") / "box_list_of_lists_norm_graphnet.pkl"
out_path = Path("box-data") / "box_list_of_lists_norm_off_graphnet.pkl"
# out_path = Path("box-data") / "box_list_of_lists_graphnet.pkl"

# Compute boxplot stats for each separate wave height range
box_list_of_lists = []
for error_list in errors_list:
    box_list = []
    for errors in error_list:
        box = {}
        box["med"] = np.nanmedian(errors)
        box["q1"] = np.nanpercentile(errors, 25)
        box["q3"] = np.nanpercentile(errors, 75)
        box["whislo"] = np.nanpercentile(errors, 5)
        box["whishi"] = np.nanpercentile(errors, 95)
        box["mean"] = np.nanmean(errors)

        box_list.append(box)

    box_list_of_lists.append(box_list)

pickle.dump(box_list_of_lists, out_path.open("wb"))

/home/jannik/anaconda3/envs/ugrid/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/jannik/anaconda3/envs/ugrid/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1555: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/tmp/ipykernel_63303/3285856305.py:16: RuntimeWarning: Mean of empty slice
  box["mean"] = np.nanmean(errors)


In [8]:
# Compute and save mae, rmse, and max error for three regions
maes = np.empty((len(diffs)))
rmses = np.empty((len(diffs)))
max_errors = np.empty((len(diffs)))

for idx, diff in enumerate(diffs):
    if idx == 2:
        diff[diff == np.inf] = np.nan
    # Convert it to cm
    diff = 100*diff
    maes[idx] = np.nanmean(diff)
    rmses[idx] = np.sqrt(np.nanmean(diff**2))
    max_errors[idx] = np.nanmax(diff)

In [9]:
np.save("box-data/stats_norm_graphnet", np.vstack((maes, rmses, max_errors)).T)

In [6]:
np.nanmedian(diffs[2])

0.0027412091819132844